<a href="https://colab.research.google.com/github/mr1er0y/Topic-Modelling-Neuro/blob/main/model_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 Рекуррентные нейронные сети являются одним из инструментов для обработки естественного языка являются. Они хорошо работают на задачах, где важна последовательность, таких как генерация текста и машинный перевод. Рекуррентные нейронные сети могут учитывать зависимости между словами на разных расстояниях и сохраняют информацию о предыдущих состояниях, что позволяет лучше понимать контекст. Однако они могут быть медленными в обучении и использовании из-за последовательной обработки данных, и могут страдать от проблемы затухания градиента при обучении на длинных последовательностях.

In [1]:
!pip install tensorflow-gpu   > /dev/null
!pip install tf-nightly  > /dev/null

  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [2]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


## Dataset Preprocessing

В первую очередь нужно скачать данные из Kaggle(у меня они хранятся на Google Drive) и произвести преобразования  
1. Токенизировать данные
2. Бинанизировать вектор тем с помощью MultiLabelBinarizer

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer

path = "drive/MyDrive/Data_arXiv/filtered_arxiv_db.csv"
SEED = 77


df = pd.read_csv(path)
df[['created_date', 'update_date']] = df[['created_date', 'update_date']].apply(pd.to_datetime)
df = df.drop(['versions', 'description', 'new_category', 'sub_category'], axis=1)
df.columns = ['id', 'title', 'authors', 'category', 'published_date', 'updated_date', 'abstract']
df["category"] = df["category"].apply(eval)
df.head()

<ipython-input-4-7f2bcc434df4>:9: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path)


,id,title,authors,category,published_date,updated_date,abstract
0,704.0033,Convergence of the discrete dipole approximati...,"Maxim A. Yurkin, Valeri P. Maltsev, Alfons G. ...","[physics.optics, physics.comp-ph]",2022-03-29,2022-03-31,We performed a rigorous theoretical converge...
1,704.0038,The discrete dipole approximation: an overview...,"Maxim A. Yurkin, Alfons G. Hoekstra","[physics.optics, physics.comp-ph]",2022-03-29,2022-03-30,We present a review of the discrete dipole a...
2,704.0275,Mapping radii of metric spaces,George M. Bergman (U.C.Berkeley),[math.MG],2008-03-28,2021-10-15,It is known that every closed curve of lengt...
3,704.0479,The affine part of the Picard scheme,T.Geisser,"[math.AG, math.KT]",2021-01-28,2021-01-29,We describe the maximal torus and maximal un...
4,704.0752,Actions for the Bosonic String with the Curved...,Davoud Kamani,[hep-th],2008-04-18,2020-08-21,At first we introduce an action for the stri...


In [5]:
# Extract the categories column as a list of lists
categories = []
for el in df["category"]:
    categories.extend(el)
categories = np.unique(categories)
num_categories = len(categories)

# Initialize the MultiLabelBinarizer and fit_transform the categories
mlb = MultiLabelBinarizer()
df['labels'] =  mlb.fit_transform(df["category"].values).tolist()

In [6]:
num_categories

170

In [8]:
def preprocessing(data):
    data = data[data['abstract'].str.contains('paper has been withdrawn') == False]
    data['abstract'] = [a.strip() for a in data['abstract']]
    data['abstract'] = data['abstract'].str.replace('\n', ' ', regex = False).str.replace('\t', ' ', regex = False).str.replace(r'\s\s+', ' ', regex = True)
    data['abstract'] = data['abstract'].str.replace('([.,!?()])', r' \1 ')
    return data

df = preprocessing(df)
df.head()

<ipython-input-8-60440652729b>:6: FutureWarning: The default value of regex will change from True to False in a future version.
  data['abstract'] = data['abstract'].str.replace('([.,!?()])', r' \1 ')


,id,title,authors,category,published_date,updated_date,abstract,labels
0,704.0033,Convergence of the discrete dipole approximati...,"Maxim A. Yurkin, Valeri P. Maltsev, Alfons G. ...","[physics.optics, physics.comp-ph]",2022-03-29,2022-03-31,We performed a rigorous theoretical convergenc...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,704.0038,The discrete dipole approximation: an overview...,"Maxim A. Yurkin, Alfons G. Hoekstra","[physics.optics, physics.comp-ph]",2022-03-29,2022-03-30,We present a review of the discrete dipole app...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,704.0275,Mapping radii of metric spaces,George M. Bergman (U.C.Berkeley),[math.MG],2008-03-28,2021-10-15,It is known that every closed curve of length ...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,704.0479,The affine part of the Picard scheme,T.Geisser,"[math.AG, math.KT]",2021-01-28,2021-01-29,We describe the maximal torus and maximal unip...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,704.0752,Actions for the Bosonic String with the Curved...,Davoud Kamani,[hep-th],2008-04-18,2020-08-21,At first we introduce an action for the string...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


## Токенизация 

`labels` - это закодированные в бинарном виде темы статей. Он состоит из 0 и 1, где 1 на i позиции обозначает, что тема i соотвествует тексту, а 0 - нет.

`input_ids` - это последовательность чисел, которые представляют отдельные токены в тексте, каждый токен преобразуется в уникальное число, которое соответствует его позиции в словаре модели BERT. Это основной входной тензор, который передается в модель.

`token_type_ids` - это последовательность чисел, которая указывает, к какому из двух предложений относится каждый токен в input_ids (0 для первого предложения и 1 для второго). Это необходимо в многих задачах, например, в задачах сравнения двух предложений, где модель должна понимать, какой токен относится к какому предложению.

`attention_mask`  это последовательность чисел, которая указывает, какие токены должны быть проигнорированы моделью в процессе обработки текста. Она состоит из 0 и 1, где 1 обозначает, что токен должен быть учтен в модели, а 0 - игнорирован.

In [9]:
df['unifed_text'] = df['title'] + '[SEP]' + df['abstract'] + '[SEP]'

In [10]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split


with tf.device('/device:GPU:0'):
    # Предварительная обработка данных
    tokenizer = Tokenizer(num_words=5000, oov_token="<OOV>")
    tokenizer.fit_on_texts(df['unifed_text'])
    word_index = tokenizer.word_index

    sequences = tokenizer.texts_to_sequences(df['unifed_text'])
    padded_sequences = pad_sequences(sequences, maxlen=350, truncating='post')

    # Подготовка данных для обучения модели
    X_train, X_test, y_train, y_test = train_test_split(padded_sequences, df['labels'], test_size=0.2,  random_state=SEED)

    # Создание модели
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(len(word_index)+1, 32),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(num_categories, activation='sigmoid')
        ])
    
    from keras import backend as K

    def recall_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

    def f1_m(y_true, y_pred):
        precision = precision_m(y_true, y_pred)
        recall = recall_m(y_true, y_pred)
        return 2*((precision*recall)/(precision+recall+K.epsilon()))


    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy', 'AUC', f1_m])


    x_train = tf.cast(X_train , dtype=tf.float32)
    y_train = tf.cast(y_train.to_list() , dtype=tf.int32)

    X_test = tf.cast(X_test , dtype=tf.float32)
    y_test = tf.cast(y_test.to_list() , dtype=tf.int32)

    history = model.fit(X_train, y_train, epochs=3, validation_data=(X_test, y_test))

    test_loss, test_acc, test_auc, test_f1 = model.evaluate(X_test, y_test)
    print('Test accuracy:', test_acc, test_auc, test_f1)


Epoch 1/3
16465/16465 [==============================] - 652s 39ms/step - loss: 429284.9062 - accuracy: 0.0032 - auc: 0.5434 - f1_m: 0.0240 - val_loss: 1322468.2500 - val_accuracy: 2.5814e-04 - val_auc: 0.5386 - val_f1_m: 0.0236
Epoch 2/3
16465/16465 [==============================] - 535s 33ms/step - loss: 3776877.5000 - accuracy: 2.6383e-04 - auc: 0.5343 - f1_m: 0.0234 - val_loss: 6908344.5000 - val_accuracy: 2.5814e-04 - val_auc: 0.5327 - val_f1_m: 0.0233
Epoch 3/3
4117/4117 [==============================] - 50s 12ms/step - loss: 19570836.0000 - accuracy: 2.5814e-04 - auc: 0.5327 - f1_m: 0.0233
Test accuracy: 0.0002581350563559681 0.532698929309845 0.023271940648555756
